In [2]:
import time
from PIL import Image
import torch
from transformers import AutoModel, AutoImageProcessor
import numpy as np

# 1. 비교 대상 모델 리스트 (7B 포함)
MODELS_TO_COMPARE = [
    "facebook/dinov3-vit7b16-pretrain-lvd1689m", # 7B 모델 (FP16/BF16 필수)
    "facebook/dinov3-vith16plus-pretrain-lvd1689m",
    "facebook/dinov3-vitl16-pretrain-lvd1689m",
    "facebook/dinov3-vitb16-pretrain-lvd1689m",
    "facebook/dinov3-vits16plus-pretrain-lvd1689m",
    "facebook/dinov3-vits16-pretrain-lvd1689m",
    "facebook/dinov3-convnext-large-pretrain-lvd1689m",
    "facebook/dinov3-convnext-base-pretrain-lvd1689m",
    "facebook/dinov3-convnext-small-pretrain-lvd1689m",
    "facebook/dinov3-convnext-tiny-pretrain-lvd1689m",
]

# 2. 이미지 로드 및 장치 설정 (이전과 동일)
IMAGE_PATH = "/home/najo/NAS/DIP/2025_ICRA_Multi_View_Robot_Pose_Estimation/dataset/franka_research3/franka_research3_pose1/Panda_dataset_1th/view1/zed_41182735_left_1756275914.742.jpg"
try:
    image = Image.open(IMAGE_PATH).convert("RGB")
except FileNotFoundError:
    # 파일이 없는 경우, 가상의 이미지로 대체하여 코드 실행이 가능하도록 함
    print(f"오류: 이미지를 찾을 수 없습니다. 경로를 확인하세요: {IMAGE_PATH}")
    image = Image.fromarray(np.zeros((224, 224, 3), dtype=np.uint8)).convert("RGB")
    print("가상의 224x224 이미지로 테스트를 진행합니다.")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"사용 장치: {DEVICE}")

# 3. 모델별 성능 비교 루프
results = []
NUM_WARMUP = 5
NUM_RUNS = 20

# VRAM 정보를 확인하여 7B 모델 로딩 가능성 경고
if DEVICE == "cuda":
    total_vram = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f"총 VRAM: {total_vram:.2f} GB. ViT-7B 로드를 위해 FP16/BF16이 필수입니다.")

print("\n--- DINOv3 AutoModel 성능 및 속도 비교 시작 ---")

for model_name in MODELS_TO_COMPARE:
    print(f"\n[모델 로딩]: {model_name}")
    
    try:
        # 3-1. 모델과 전처리기(Image Processor) 로드
        processor = AutoImageProcessor.from_pretrained(model_name)
        
        # BF16으로 모델 로드 (7B 포함 모든 모델에 대해 최적의 추론 속도 및 메모리 사용을 위해)
        model = AutoModel.from_pretrained(
            model_name, 
            torch_dtype=torch.bfloat16,  # BF16 사용
            low_cpu_mem_usage=True      
        ).to(DEVICE)
        model.eval()

    except RuntimeError as e:
        if "CUDA out of memory" in str(e) or "Input type (float) and bias type" in str(e):
             # 메모리 부족 또는 이전 FP32 입력 오류 처리 (BF16으로 해결되었을 가능성이 높음)
            print("❌ 로드 실패: VRAM 부족 (7B 모델일 가능성 높음) 또는 타입 불일치. 다음 모델로 넘어갑니다.")
            continue
        else:
             print(f"❌ 로드 실패 (RuntimeError): {e}")
             continue
    except Exception as e:
        print(f"❌ 로드 실패 (AutoModel 문제): {e}")
        continue
    
    # 3-2. 이미지 전처리
    inputs = processor(images=image, return_tensors="pt")
    
    # [수정된 핵심 로직]: 입력 텐서의 타입을 모델의 타입(BF16)과 일치시키고 GPU로 전송
    for key in inputs:
        inputs[key] = inputs[key].to(torch.bfloat16).to(DEVICE) 
        
    # 3-3. 워밍업 (GPU/CPU 캐시 최적화)
    with torch.no_grad():
        for _ in range(NUM_WARMUP):
            _ = model(**inputs)
    
    # 3-4. 실제 추론 시간 측정 (이전과 동일)
    latencies = []
    
    if DEVICE == "cuda":
        torch.cuda.synchronize()
        
    with torch.no_grad():
        for _ in range(NUM_RUNS):
            start_run = time.time()
            outputs = model(**inputs)
            if DEVICE == "cuda":
                torch.cuda.synchronize()
            end_run = time.time()
            latencies.append(end_run - start_run)

    # 3-5. 결과 정리
    avg_latency_ms = (sum(latencies) / NUM_RUNS) * 1000
    
    feature_tensor = outputs.last_hidden_state if 'last_hidden_state' in outputs else outputs[0]
    feature_shape = tuple(feature_tensor.shape)
    
    results.append({
        "Model": model_name.split('/')[-1],
        "Avg Latency (ms)": f"{avg_latency_ms:.3f}",
        "Feature Shape": feature_shape
    })
    
    print(f"  ✅ 로딩 성공!")
    print(f"  평균 지연 시간: {avg_latency_ms:.3f} ms")
    print(f"  추출 특징 형태: {feature_shape}")
    
    # 메모리 해제 (7B 모델 로드 후 메모리 부족 방지)
    del model
    if DEVICE == "cuda":
        torch.cuda.empty_cache()


# 4. 최종 결과 출력 (이전과 동일)
print("\n--- 최종 비교 결과 ---")
if results:
    # 모델 크기별 순서 (7B 추가)
    order = [
        "vit7b16", "vith16plus", "vitl16", "vitb16", "vits16plus", "vits16", 
        "convnext-large", "convnext-base", "convnext-small", "convnext-tiny"
    ]
    
    def get_order_key(model_id):
        full_name = model_id['Model']
        if 'vit' in full_name:
            name_part = full_name.split('-')[1]
        elif 'convnext' in full_name:
            name_parts = full_name.split('-')
            name_part = f"{name_parts[1]}-{name_parts[2]}" 
        else:
            return len(order) 

        try:
            return order.index(name_part)
        except ValueError:
            return len(order) 
            
    sorted_results = sorted(results, key=get_order_key)

    print(f"{'Model':<40} | {'Avg Latency (ms)':>20} | {'Feature Shape':>20}")
    print("-" * 85)
    
    for res in sorted_results:
        shape_str = str(res['Feature Shape'])
        print(f"{res['Model']:<40} | {res['Avg Latency (ms)']:>20} | {shape_str:>20}")
else:
    print("비교할 결과를 얻지 못했습니다.")

사용 장치: cuda
총 VRAM: 23.57 GB. ViT-7B 로드를 위해 FP16/BF16이 필수입니다.

--- DINOv3 AutoModel 성능 및 속도 비교 시작 ---

[모델 로딩]: facebook/dinov3-vit7b16-pretrain-lvd1689m


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

  ✅ 로딩 성공!
  평균 지연 시간: 66.719 ms
  추출 특징 형태: (1, 201, 4096)

[모델 로딩]: facebook/dinov3-vith16plus-pretrain-lvd1689m


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

...

  ✅ 로딩 성공!
  평균 지연 시간: 7.463 ms
  추출 특징 형태: (1, 50, 1024)

[모델 로딩]: facebook/dinov3-convnext-small-pretrain-lvd1689m


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

  ✅ 로딩 성공!
  평균 지연 시간: 7.484 ms
  추출 특징 형태: (1, 50, 768)

[모델 로딩]: facebook/dinov3-convnext-tiny-pretrain-lvd1689m


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

  ✅ 로딩 성공!
  평균 지연 시간: 4.092 ms
  추출 특징 형태: (1, 50, 768)

--- 최종 비교 결과 ---
Model                                    |     Avg Latency (ms) |        Feature Shape
-------------------------------------------------------------------------------------
dinov3-vit7b16-pretrain-lvd1689m         |               66.719 |       (1, 201, 4096)
dinov3-vith16plus-pretrain-lvd1689m      |               21.643 |       (1, 201, 1280)
dinov3-vitl16-pretrain-lvd1689m          |               19.433 |       (1, 201, 1024)
dinov3-vitb16-pretrain-lvd1689m          |                9.939 |        (1, 201, 768)
dinov3-vits16plus-pretrain-lvd1689m      |                8.270 |        (1, 201, 384)
dinov3-vits16-pretrain-lvd1689m          |                8.102 |        (1, 201, 384)
dinov3-convnext-large-pretrain-lvd1689m  |                7.411 |        (1, 50, 1536)
dinov3-convnext-base-pretrain-lvd1689m   |                7.463 |        (1, 50, 1024)
dinov3-convnext-small-pretrain-lvd1689m  |              

In [1]:
import torch
import numpy as np
from PIL import Image
from transformers import AutoModel, AutoImageProcessor
import matplotlib.pyplot as plt

def extract_attention_map(model_id, image, device):
    print(f"\n[어텐션 맵 추출]: {model_id}")
    
    # 1. 모델과 프로세서 로드 (output_attentions=True 설정)
    try:
        processor = AutoImageProcessor.from_pretrained(model_id)
        # AutoModel 대신 특정 ViT 클래스를 사용하거나, AutoModel의 설정(config)를 수정해야 합니다.
        # DINOv3는 AutoModel로 로드 가능하지만, Hugging Face 모델이 output_attentions=True를 기본적으로 제공하지 않을 수 있습니다.
        
        # 안전을 위해 AutoModel.from_pretrained에서 직접 설정 전달
        model = AutoModel.from_pretrained(model_id, output_attentions=True)
        model.to(device)
        model.eval()

    except Exception as e:
        print(f"로딩 실패: {e}")
        return None, None

    # 2. 이미지 전처리
    inputs = processor(images=image, return_tensors="pt").to(device)
    
    # 3. 추론 및 어텐션 가중치 추출
    with torch.no_grad():
        outputs = model(**inputs)
    
    # DINOv3 ViT 모델은 'attentions' 키를 통해 가중치 튜플을 반환합니다.
    # 각 요소는 (Batch, Head, Seq_Len, Seq_Len) 형태입니다.
    # 일반적으로 마지막 레이어의 어텐션 가중치를 사용합니다.
    attention_weights_list = outputs.attentions
    
    if not attention_weights_list:
        print("경고: output_attentions=True 옵션이 적용되지 않았거나 지원되지 않습니다.")
        return None, None

    # 마지막 레이어의 어텐션 가중치 (Attention Weights from the last layer)
    # shape: [Batch=1, Num_Heads, Seq_Len, Seq_Len]
    last_layer_attention = attention_weights_list[-1][0].cpu().numpy()
    
    return last_layer_attention, model_id.split('/')[-1]

def visualize_attention(attention_map, image, model_name):
    # CLS 토큰 (첫 번째 인덱스)에서 다른 모든 토큰으로의 어텐션 가중치를 사용
    # attention_map shape: [Num_Heads, Seq_Len, Seq_Len]
    # CLS 토큰에서 공간 토큰으로의 어텐션 가중치 (Seq_Len - 1)
    cls_attention = attention_map[:, 0, 1:].mean(axis=0) # [Seq_Len - 1] (현재 200)
    
    seq_len_spatial = cls_attention.shape[0] # 200

    # 1. 그리드 사이즈 추정: 200에 가장 가까운 정수 제곱수 (14*14=196, 15*15=225)를 사용하여 근사
    # DINOv3 ViT-B16의 표준 해상도는 14x14=196 이므로, 200개 중 196개만 사용하도록 강제함
    grid_size = 14 # 표준 ViT-B16 패치 수의 제곱근
    target_seq_len = grid_size * grid_size # 196
    
    if seq_len_spatial > target_seq_len:
        # 시퀀스 길이가 196보다 크면 (현재 200), 초과된 토큰을 버립니다.
        att_map_1d = cls_attention[:target_seq_len]
        print(f"  주의: 시퀀스 길이 {seq_len_spatial} 중 {target_seq_len}개 토큰만 사용.")
    elif seq_len_spatial < target_seq_len:
        print(f"시각화 오류: 시퀀스 길이가 196 미만입니다.")
        return
    else:
        att_map_1d = cls_attention
        
    # 2. 2D 맵으로 재구성 및 리사이징
    att_map_2d = att_map_1d.reshape(grid_size, grid_size)
    
    # 3. 시각화
    plt.figure(figsize=(6, 5))
    plt.imshow(image)
    
    # 히트맵을 원본 이미지 크기에 맞춰 부드럽게 오버레이
    plt.imshow(att_map_2d, cmap='viridis', alpha=0.6, 
               extent=[0, image.width, image.height, 0], interpolation='bilinear')
    
    plt.title(f'Attention Map (CLS Token): {model_name}')
    plt.axis('off')
    plt.show()
    
# --- 실행 예시 ---
# (이전에 로드한 image와 DEVICE 변수가 정의되어 있다고 가정)
IMAGE_PATH = "/home/najo/NAS/DIP/2025_ICRA_Multi_View_Robot_Pose_Estimation/dataset/franka_research3/franka_research3_pose1/Panda_dataset_1th/view1/zed_41182735_left_1756275914.742.jpg"
image = Image.open(IMAGE_PATH).convert("RGB")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# 1. ViT-L16 분석
att_map_L, name_L = extract_attention_map("facebook/dinov3-vit7b16-pretrain-lvd1689m", image, DEVICE)
if att_map_L is not None:
    visualize_attention(att_map_L, image, name_L)

# 2. ViT-B16 분석
att_map_B, name_B = extract_attention_map("facebook/dinov3-vitb16-pretrain-lvd1689m", image, DEVICE)
if att_map_B is not None:
    visualize_attention(att_map_B, image, name_B)


[어텐션 맵 추출]: facebook/dinov3-vit7b16-pretrain-lvd1689m


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

로딩 실패: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 23.57 GiB of which 30.44 MiB is free. Including non-PyTorch memory, this process has 23.52 GiB memory in use. Of the allocated memory 23.27 GiB is allocated by PyTorch, and 943.50 KiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

[어텐션 맵 추출]: facebook/dinov3-vitb16-pretrain-lvd1689m


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

AcceleratorError: CUDA error: out of memory
Search for `cudaErrorMemoryAllocation' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
